# Model Training (NLP)

This notebook trains 5 transformer-based NLP models:
- BERT
- ClinicalBERT
- DistilBERT
- BioBERT
- ALBERT

In [1]:
import os
import math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    logging as tf_logging,
    set_seed
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

set_seed(42)
torch.manual_seed(42)

BASE_DIR = Path.cwd().parents[1]
NLP_FEATURE_DIR = BASE_DIR / "features" / "05 - Dataset 1 (NLP)"
PROC_OUT = BASE_DIR / "results" / "05 - Dataset 1 (NLP)"
MODEL_OUT = BASE_DIR / "models" / "05 - Dataset 1 (NLP)"
FIG_OUT = BASE_DIR / "figures" / "05 - Dataset 1 (NLP)"

for p in [PROC_OUT, MODEL_OUT, FIG_OUT]:
    p.mkdir(parents=True, exist_ok=True)

NUM_EPOCHS = 5
PER_DEVICE_TRAIN_BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH_SIZE = 8
MAX_LENGTH = 128
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

## Load Train and Test Sets

In [2]:
train_csv = NLP_FEATURE_DIR / "train.csv"
test_csv  = NLP_FEATURE_DIR / "test.csv"

if not train_csv.exists() or not test_csv.exists():
    raise FileNotFoundError(f"Train/test CSVs not found in {NLP_FEATURE_DIR}")

train_df = pd.read_csv(train_csv)
test_df  = pd.read_csv(test_csv)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("\nSample:")
display(train_df.head(2))

for c in ["Student Information", "Depression Label"]:
    if c not in train_df.columns:
        raise ValueError(f"Column {c} missing in {train_csv}")
    if c not in test_df.columns:
        raise ValueError(f"Column {c} missing in {test_csv}")

Train shape: (1617, 2)
Test shape: (405, 2)

Sample:


,Student Information,Depression Label
0,"The student is around 23-26 years old, male, s...",Severe
1,"The student is around 18-22 years old, male, s...",Severe


## Encode Target Labels

In [3]:
le = LabelEncoder()
train_df["label_enc"] = le.fit_transform(train_df["Depression Label"].astype(str))
test_df["label_enc"]  = le.transform(test_df["Depression Label"].astype(str))

num_labels = len(le.classes_)
print("Classes:", list(le.classes_))
print("Num labels:", num_labels)

train_ds = Dataset.from_pandas(train_df[["Student Information", "label_enc"]].rename(columns={"Student Information":"text", "label_enc":"label"}))
test_ds  = Dataset.from_pandas(test_df[["Student Information", "label_enc"]].rename(columns={"Student Information":"text", "label_enc":"label"}))

print(train_ds)
print(test_ds)

Classes: ['Mild', 'Minimal', 'Moderate', 'Moderately Severe', 'Severe']
Num labels: 5
Dataset({
    features: ['text', 'label'],
    num_rows: 1617
})
Dataset({
    features: ['text', 'label'],
    num_rows: 405
})


## Models List

In [4]:
MODELS = [
    ("bert", "bert-base-uncased"),
    ("clinical_bert", "emilyalsentzer/Bio_ClinicalBERT"),
    ("distilbert", "distilbert-base-uncased"),
    ("biobert", "dmis-lab/biobert-base-cased-v1.1"),
    ("albert", "albert-base-v2"),
]

print("Models to run:", [m[0] for m in MODELS])

Models to run: ['bert', 'clinical_bert', 'distilbert', 'biobert', 'albert']


## Helper Functions

In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average="weighted", zero_division=0)
    rec = recall_score(labels, preds, average="weighted", zero_division=0)
    f1 = f1_score(labels, preds, average="weighted", zero_division=0)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

## Training Loop

In [6]:
RUN_MODELS = MODELS

tf_logging.set_verbosity_error()

for slug, hf_name in RUN_MODELS:
    print("\n" + "="*80)
    print(f"Starting training: {slug}  (HF model: {hf_name})")
    print("="*80)
    
    model_out_dir = MODEL_OUT / slug
    fig_out_dir = FIG_OUT / slug
    proc_out_dir = PROC_OUT
    model_out_dir.mkdir(parents=True, exist_ok=True)
    fig_out_dir.mkdir(parents=True, exist_ok=True)
    proc_out_dir.mkdir(parents=True, exist_ok=True)

    print("Loading tokenizer and model...")
    tokenizer = AutoTokenizer.from_pretrained(hf_name, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(hf_name, num_labels=num_labels)

    def tokenize_fn(batch):
        return tokenizer(batch["text"], truncation=True, padding=False, max_length=MAX_LENGTH)

    tokenized_train = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
    tokenized_test  = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir=str(model_out_dir),
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        disable_tqdm=False,
        load_best_model_at_end=False,
        metric_for_best_model="f1",
        greater_is_better=True,
        fp16=False,
        push_to_hub=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    print("Beginning training...")
    train_result = trainer.train()
    trainer.save_model(str(model_out_dir))
    print("Model saved to:", model_out_dir)

    print("Running final evaluation on test set...")
    metrics = trainer.evaluate(eval_dataset=tokenized_test)
    print("Final metrics:", metrics)

    logs = trainer.state.log_history
    epoch_nums = []
    epoch_acc = []
    epoch_prec = []
    epoch_rec = []
    epoch_f1 = []
    for entry in logs:
        if "eval_accuracy" in entry:
            epoch_nums.append(entry.get("epoch"))
            epoch_acc.append(entry.get("eval_accuracy"))
            epoch_prec.append(entry.get("eval_precision"))
            epoch_rec.append(entry.get("eval_recall"))
            epoch_f1.append(entry.get("eval_f1"))
    if not epoch_nums:
        epoch_nums = [i+1 for i in range(NUM_EPOCHS)]
        epoch_acc = [metrics.get("eval_accuracy")] * NUM_EPOCHS
        epoch_prec = [metrics.get("eval_precision")] * NUM_EPOCHS
        epoch_rec = [metrics.get("eval_recall")] * NUM_EPOCHS
        epoch_f1 = [metrics.get("eval_f1")] * NUM_EPOCHS

    results_df = pd.DataFrame({
        "epoch": epoch_nums,
        "accuracy": epoch_acc,
        "precision": epoch_prec,
        "recall": epoch_rec,
        "f1": epoch_f1
    })
    results_csv = proc_out_dir / f"{slug}_results.csv"
    results_df.to_csv(results_csv, index=False)
    print("Saved epoch metrics CSV ->", results_csv)
    
    summary_out = proc_out_dir / f"{slug}_final_metrics.csv"
    pd.DataFrame([metrics]).to_csv(summary_out, index=False)
    print("Saved final metrics ->", summary_out)

    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(epoch_nums, epoch_acc, marker='o', label='Accuracy')
    ax.plot(epoch_nums, epoch_f1, marker='o', label='F1')
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Score")
    ax.set_title(f"{slug} — Accuracy/F1 vs Epoch")
    ax.set_xticks(epoch_nums)
    ax.legend()
    acc_fig_path = fig_out_dir / f"{slug}_accuracy_epoch.png"
    fig.savefig(acc_fig_path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Saved accuracy vs epoch ->", acc_fig_path)

    print("Computing confusion matrix on test set...")
    preds_output = trainer.predict(tokenized_test)
    pred_labels = np.argmax(preds_output.predictions, axis=-1)
    true_labels = preds_output.label_ids
    cm = confusion_matrix(true_labels, pred_labels)
    cm_fig, ax = plt.subplots(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(f"{slug} — Confusion Matrix")
    cm_path = fig_out_dir / f"{slug}_confusion.png"
    cm_fig.savefig(cm_path, dpi=300, bbox_inches="tight")
    plt.close(cm_fig)
    print("Saved confusion matrix ->", cm_path)

    mapping_path = model_out_dir / "label_mapping.csv"
    pd.DataFrame({"label": list(le.classes_), "enc": list(range(len(le.classes_)))}).to_csv(mapping_path, index=False)
    print("Saved label mapping ->", mapping_path)
    
    print(f"Finished model: {slug}")


Starting training: bert  (HF model: bert-base-uncased)
Loading tokenizer and model...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

d:\Programming\Projects\Depression Severity Assessment\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ryo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Model saved to: d:\Programming\Projects\Depression Severity Assessment\models\05 - Dataset 1 (NLP)\albert
Running final evaluation on test set...


d:\Programming\Projects\Depression Severity Assessment\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Final metrics: {'eval_loss': 1.354619026184082, 'eval_accuracy': 0.4123456790123457, 'eval_precision': 0.412117884578736, 'eval_recall': 0.4123456790123457, 'eval_f1': 0.3930856324660449, 'eval_runtime': 61.897, 'eval_samples_per_second': 6.543, 'eval_steps_per_second': 0.824, 'epoch': 5.0}
Saved epoch metrics CSV -> d:\Programming\Projects\Depression Severity Assessment\results\05 - Dataset 1 (NLP)\albert_results.csv
Saved final metrics -> d:\Programming\Projects\Depression Severity Assessment\results\05 - Dataset 1 (NLP)\albert_final_metrics.csv
Saved accuracy vs epoch -> d:\Programming\Projects\Depression Severity Assessment\figures\05 - Dataset 1 (NLP)\albert\albert_accuracy_epoch.png
Computing confusion matrix on test set...


d:\Programming\Projects\Depression Severity Assessment\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved confusion matrix -> d:\Programming\Projects\Depression Severity Assessment\figures\05 - Dataset 1 (NLP)\albert\albert_confusion.png
Saved label mapping -> d:\Programming\Projects\Depression Severity Assessment\models\05 - Dataset 1 (NLP)\albert\label_mapping.csv
Finished model: albert


## Final Results

In [7]:
agg = []
for slug, hf_name in MODELS:
    summary_out = PROC_OUT / f"{slug}_final_metrics.csv"
    if summary_out.exists():
        dfm = pd.read_csv(summary_out)
        dfm["model"] = slug
        agg.append(dfm)
    else:
        print("Summary missing for", slug)

if agg:
    combined = pd.concat(agg, ignore_index=True, sort=False)
    display(combined[["model", "eval_accuracy", "eval_precision", "eval_recall", "eval_f1", "eval_loss"]])
    combined.to_csv(BASE_DIR / "Summary" / "results" / "Results Summary (NLP)" / "dataset1_results_summary.csv", index=False)
    print("Saved combined summary ->", BASE_DIR / "Summary" / "results" / "Results Summary (NLP)" / "dataset1_results_summary.csv")
else:
    print("No final metrics found.")

,model,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_loss
0,bert,0.429630,0.440107,0.429630,0.413036,1.341254
1,clinical_bert,0.434568,0.433070,0.434568,0.408489,1.346020
2,distilbert,0.432099,0.435941,0.432099,0.409886,1.345803
3,biobert,0.407407,0.413351,0.407407,0.393020,1.352443
4,albert,0.412346,0.412118,0.412346,0.393086,1.354619


Saved combined summary -> d:\Programming\Projects\Depression Severity Assessment\Summary\results\Results Summary (NLP)\dataset1_results_summary.csv
